In [ ]:
import pandas as pd
import sqlite3 as sql

In [14]:
mydb = 'Abstracts_aug12.db'

In [ ]:
jsonFile = "json42.json"

# Helper Functions to inspect database tables and json file

In [ ]:
def jsonDF(jsonFile):
    f = open(jsonFile, "r+")
    return pd.read_json(f, orient='index')
    

In [ ]:
def getContents():
    '''
    : param NONE
    : output : Returns a json dictionary of the table names, entry counts, and links to tables 
                of all table names in the database
    ''' 
    with sql.connect(mydb) as con:
    
        cursor = con.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        mytables = (cursor.fetchall())
        myt = [t[0] for t in mytables]
        
        return myt
            

In [ ]:
def sqlCMDToPD(table, 
               db):
    '''Take a sql db and return as a readable pandas DataFrame
       : param db : str. Name of db. (ie. 'Abstracts.db'
       : param sqlcmd : str. Sqlite3 cmd to execute. 
               default: "SELECT * FROM Abstracts" 
                           > select all from Abstracts table
    '''
    
    #connect to a db
    with sql.connect(db) as con:
        
        #run command
        sqlcmd = "SELECT * FROM '%s'" %table
        df = pd.read_sql_query(sqlcmd, con)
        
        # Check resulting pandas DF shape
        print df.shape
        
        return df


In [ ]:
t = getContents()
t

In [ ]:
c = sqlCMDToPD("CONFERENCES", mydb)
t = sqlCMDToPD("ABSTRACTSTOTAL", mydb)
pap = sqlCMDToPD("PAPER", mydb)
pub = sqlCMDToPD("PUBLICATIONS", mydb)
k = sqlCMDToPD("KEYS", mydb)

au = sqlCMDToPD("AUTHORS", mydb)
aff = sqlCMDToPD("AFFILIATIONS", mydb)

pk = sqlCMDToPD("PAPERKEY", mydb)
ap = sqlCMDToPD("AFFILIATIONPAPER", mydb)
pa = sqlCMDToPD("PAPERAUTHOR", mydb)
t.tail()


#  Functions using sqlite3 and python to insert and delete and update tables.


In [ ]:
def insert_toTable(db, df, table = 'ABSTRACTSTOTAL'):
    '''Check to insert a new record into a database table, inserts if does not exist
    param  db str : Database name to connect to
    param df pandas dataframe : dataframe being inspected for entry
    param table str : Table Name to insert into, if does not exist will create
    output : new entry inserted
    '''
    with sql.connect(db) as con:
        df.to_sql(table, con, flavor='sqlite', 
                      schema=None, if_exists='append',
                      index=False, index_label=None,
                      chunksize=None, dtype=None)
            
        print("Records %s inserted"%table)
    
def insertcheckRecord(db, df, table = 'CONFERENCES', un = 'confName' ):
    '''Check to insert a new record into a database table, inserts if does not exist
    param  db str : Database name to connect to
    param df pandas dataframe : dataframe being inspected for entry
    param table str : Table Name to insert into, if does not exist will create
    param un str : unique column to check for entry to create a new pk, if not will just append
    '''
    t = sqlCMDToPD(table, mydb)
    if df[un][0] not in t[un].unique():
        insert_toTable(db, df[un], table)
    else:
        print(" %s already exists, try upserting with key value or deleting" %df[un][0])

    
def insertcheckRecordTWO(db, df, table = 'PUBLICATIONS', un = 'confName', un1 = 'year' ):
    '''Check to insert a new record into a database table, inserts if does not exist, checks for multiple
    entries as unique
    param  db str : Database name to connect to
    param df pandas dataframe : dataframe being inspected for entry
    param table str : Table Name to insert into, if does not exist will create
    param un str : unique column to check for entry to create a new pk, if not will just append
    param un1 str : unique column2 to check for entry to create a new pk, if fail un, if not will just append
    '''
    t = sqlCMDToPD(table, mydb)
    if df[un][0] not in t[un].unique():
        print df[un][0], 'is new'
        insert_toTable(db, df[[un, un1]], table)
    else:
        conf = df[un][0]
        tmp = t.query('@conf == confName') 
        
        if df[un1][0] not in tmp[un1].unique():
            print (df[[un, un1]].values), 'is a new entry'
            insert_toTable(db, df[[un, un1]], table)
        else:
            print(" %s already exists, try upserting with key value or deleting" %df[[un, un1]].values)


def insertValues(db, table, value1, value2):
    '''Insert a new record by value into a database table
    param  db str : Database name to connect to
    param table str : Table Name to insert into, if does not exist will create
    param value str : unique value entered into table
    '''
    with sql.connect(db) as con:
        try:
            con.execute("INSERT INTO {tn} VALUES('%s','%s')".format(tn=table)%(value1, value2))
            print('%s %s inserted into %s')%(value1, value2, table)
        except:
            con.execute("INSERT INTO {tn} VALUES(Null,'%s')".format(tn=table)%(value2))
            print('%s inserted into %s, single entry')%(value2, table)

        
def enterValueCheck_nested(db, table, values, cn):
    '''Check to insert a new record into a database table, inserts if does not exist
    param  db str : Database name to connect to
    param table str : Table Name to insert into, if does not exist will create
    param values python series : series being parsed and formated to inspection and entry into table
    param cn str : column name to check for entry to create a new pk
    '''
    keys = []
    tableDF = sqlCMDToPD(table, db)
    for i, ky in enumerate(values):
        for key in ky.split(','):
            if key not in tableDF[cn].unique():
                print key, 'is new'
                insertValues(db, table, None, key)
            else:
                print key, 'already exists in table'
            
            keys.append(key)
        return keys

def compositeCreation(db, table1, col, values, parentID, comptable):
    '''Creating Composite Tables 
    First, find the values needed to insert from first table based on query
    then insert each (parentID, value) pair into the composite table 
    param  db str : Database name to connect to
    param table1 str : Table Name to query for multiple values
    param col str : column name of value to retrieve iteratively 
    param values list : list of values to insert into the composite
    param parentID int : integer value (Primary Key Value) of parent table to enter
    param comptable str : Table Name of composite table
    '''
    t = sqlCMDToPD(table1, db)
    tmp = t.query('{cn} in @values'.format(cn = col))[col]
    for v in tmp.values:
        print v, paperK
        insertValues(db, comptable, parentID, v)
        
def getPK(db, table, pkCol):
    '''retrieve the PRIMARY KEY
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param pkcol str : primary column name being used, 
    '''
    with sql.connect(db) as c:
        cursor = c.cursor()
        cursor.execute("SELECT {idf} FROM {tn} ORDER BY {idf} DESC LIMIT 1".format(tn=table, idf=pkCol))
        key = cursor.fetchone()[0]

        return key
    
def deleteRowPK(db, table, pkcol, entryID):
    '''Deleting a Record by PRIMARY KEY
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param pkcol str : primary column name being used, 
    param entryID int : integer value (Primary Key Value) to delete from table
    '''
    with sql.connect(db) as con:
        
        con.execute("DELETE FROM {tn} WHERE {idf}={my_id}".format(tn=table, idf=pkcol, my_id=entryID))

        con.commit()

def deleteRowOTHER(db, table, cn, entry):
    '''Deleting a Record
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param cn str : column name being used for deletion comparason (if no PK column, ie, abstracts Total)
    param entry str : the str to be used to find and remove records (removes all records
    '''
    with sql.connect(db) as con:
        
        con.execute("DELETE FROM {tn} WHERE {idf}='%s'".format(tn=table, idf=cn)%entry)

        con.commit()
        
def entryintotables(db, jsonfile):
    '''Inserting a Record from a JsonFile
    param  db str : Database name to connect to
    param jsonfile str : name of Json File to be read into the database
    '''
    f = open(jsonfile, "r+")
    jdf = pd.read_json(f, orient='index')
    #TOTALABSTRACTS, check and then insert if needed, uniqueness based on Abstract column
    insert_toTable(db, jdf, table = 'ABSTRACTSTOTAL')
    


    #renaming of columns
    jdf.rename(columns = {'Conf':'confName'}, inplace= True)
    jdf.rename(columns = {'Author affiliation' : 'affiliation'}, inplace = True)
    jdf.rename(columns = {'Authors' : 'authors'}, inplace = True)
    
    #CONFERENCES, check and then insert if needed, uniqueness based on Abstract column
    insertcheckRecord(db, jdf, table = 'CONFERENCES', un = 'confName' )
    
    #PUBLICATIONS, check and then insert if needed, uniqueness based on Abstract column
    insertcheckRecordTWO(db, jdf, table = 'PUBLICATIONS', un = 'confName', un1 = 'year' )
    
    #AFFILIATIONS
    insertcheckRecord(db = db, df = jdf, table = 'AFFILIATIONS', un = 'affiliation')
    
    #For the nested: authors, keywords, and need to reparse/reformat, also to show numerous ways to insert:
    #KEYS
    keys = enterValueCheck_nested(db=db, table = 'KEYS', values = jdf.terms, cn = 'keyword')
    
    #AUTHORS
    authors = enterValueCheck_nested(db=db, table = 'AUTHORS', values = jdf.authors, cn = 'authorName')
    
    #PAPER
    jdf.rename(columns = {'year' : 'pubYear'}, inplace = True)
    insert_toTable(mydb, jdf, 'PAPER')
    paperID  = getPK(db, 'PAPER', 'paperID') 
    
    #COMPOSITE TABLE UPDATES
    #PAPERKEY
    compositeCreation(db, 'KEYS', 'keyword', keys, paperID, 'PAPERKEY')
    
    #PAPERAUTHOR
    compositeCreation(db, 'AUTHORS', 'authorName', authors, paperID, 'PAPERAUTHOR')
    
    #AFFILIATIONPAPER
    affilationID = getPK(db, 'AFFILIATIONS', 'affilID' )
    insertValues(mydb, "AFFILIATIONPAPER", paperID, affilationID)
    
    
    return jdf, keys, authors

In [ ]:
j, keylist, authorslist = entryintotables(mydb, jsonFile )

In [ ]:
c = sqlCMDToPD("CONFERENCES", mydb)
t = sqlCMDToPD("ABSTRACTSTOTAL", mydb)
pap = sqlCMDToPD("PAPER", mydb)
pub = sqlCMDToPD("PUBLICATIONS", mydb)
k = sqlCMDToPD("KEYS", mydb)

au = sqlCMDToPD("AUTHORS", mydb)
aff = sqlCMDToPD("AFFILIATIONS", mydb)

pk = sqlCMDToPD("PAPERKEY", mydb)
ap = sqlCMDToPD("AFFILIATIONPAPER", mydb)
pa = sqlCMDToPD("PAPERAUTHOR", mydb)
pa.tail()

In [ ]:
#t.tail()
jtest = pd.read_json(jsonFile, orient='index')
jtest

In [ ]:

deleteRowOTHER(mydb, "PAPER", 'title', 'Monty Python and the Holy Grail' )
pap2 = sqlCMDToPD("PAPER", mydb)
pap2.tail()

In [ ]:
keys = sqlCMDToPD('KEYS', mydb)
keys.tail()

In [ ]:
deleteRowPK(mydb, 'KEYS', 'keyID', 5562)
deleteRowPK(mydb, 'KEYS', 'keyID', 5561)
deleteRowPK(mydb, 'KEYS', 'keyID', 5560)
deleteRowPK(mydb, 'KEYS', 'keyID', 5558)
deleteRowPK(mydb, 'AUTHORS', 'authorID', 1950)
deleteRowPK(mydb, 'AUTHORS', 'authorID', 1951)
deleteRowPK(mydb, 'AUTHORS', 'authorID', 1952)
deleteRowPK(mydb, 'AUTHORS', 'authorID', 1950)
deleteRowPK(mydb, 'AFFILIATIONS', 'affilID', 831)
keys = sqlCMDToPD('KEYS', mydb)
au = sqlCMDToPD('AUTHORS', mydb)
aff = sqlCMDToPD('AFFILIATIONS', mydb)
aff.tail()

# Looking at some other sqlite functions

In [ ]:
def retrievals(db, table_name, column_2, column_3, id_column, some_id ):
    with sql.connect(db) as con:
        c =  cursor = con.cursor()
        # 1) Contents of all columns for row that match a certain value in 1 column
        c.execute('SELECT * FROM {tn} WHERE {cn}="ECSA"'.\
                  format(tn=table_name, cn=column_2))
        all_rows = c.fetchall()
        print('1):', all_rows)

    # 2) Value of a particular column for rows that match a certain value in column_1
        c.execute('SELECT ({coi}) FROM {tn} WHERE {cn}="ECSA"'.\
                  format(coi=column_2, tn=table_name, cn=column_2))
        all_rows = c.fetchall()
        print('2):', all_rows)

    # 3) Value of 2 particular columns for rows that match a certain value in 1 column
        c.execute('SELECT {coi1},{coi2} FROM {tn} WHERE {coi1}="ECSA"'.\
                  format(coi1=column_2, coi2=column_3, tn=table_name, cn=column_2))
        all_rows = c.fetchall()
        print('3):', all_rows)

    # 4) Selecting only up to 10 rows that match a certain value in 1 column
        c.execute('SELECT * FROM {tn} WHERE {cn}="ECSA" LIMIT 10'.\
                  format(tn=table_name, cn=column_2))
        ten_rows = c.fetchall()
        print('4):', ten_rows)

    # 5) Check if a certain ID exists and print its column contents
        c.execute("SELECT * FROM {tn} WHERE {idf}={my_id}".\
                  format(tn=table_name, cn=column_2, idf=id_column, my_id=some_id))
        id_exists = c.fetchone()
        if id_exists:
            print('5): {}'.format(id_exists))
        else:
            print('5): {} does not exist'.format(some_id))


In [ ]:
retrievals(mydb, 'CONFERENCES', 'confName', 'confID', 'confID', 3 )

In [ ]:
f = paperPK(mydb, 'PAPER', 'paperID')

In [ ]:
print keylist


# SQLite 

In [ ]:
#Remember to put PRAGMA foreign_key = ON


In [ ]:
sql_FK = """
SELECT sql
  FROM (
        SELECT sql sql, type type, tbl_name tbl_name, name name
          FROM sqlite_master
         UNION ALL
        SELECT sql, type, tbl_name, name
          FROM sqlite_temp_master
       )
WHERE type != 'meta'
   AND sql NOTNULL
   AND name NOT LIKE 'sqlite_%'
 ORDER BY substr(type, 2, 1), name"""

def getTablesFK(cmd = sql_FK, db):
    '''
    : param NONE
    : output : Returns a json dictionary of the table names, entry counts, and links to tables 
                of all table names in the database
    ''' 
    with sql.connect(db) as con:
    
        df = pd.read_sql_query(cmd, con)
        df_splitter = lambda x: pd.Series([i for i in x.split(',')])
        df = df['sql'].apply(foo)
        return df
        

In [ ]:
dftest = getFKS(sqlcmd_fk, mydb)

In [ ]:
dftest

In [ ]:
c = sqlCMDToPD("CONFERENCES", mydb)
t = sqlCMDToPD("ABSTRACTSTOTAL", mydb)
pap = sqlCMDToPD("PAPER", mydb)
pub = sqlCMDToPD("PUBLICATIONS", mydb)
k = sqlCMDToPD("KEYS", mydb)

au = sqlCMDToPD("AUTHORS", mydb)
aff = sqlCMDToPD("AFFILIATIONS", mydb)

pk = sqlCMDToPD("PAPERKEY", mydb)
ap = sqlCMDToPD("AFFILIATIONPAPER", mydb)
pa = sqlCMDToPD("PAPERAUTHOR", mydb)


In [ ]:
def deleteRowPK(db, table, pkcol, entryID):
    '''Deleting a Record by PRIMARY KEY
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param pkcol str : primary column name being used, 
    param entryID int : integer value (Primary Key Value) to delete from table
    '''
    with sql.connect(db) as con:
        con.execute("PRAGMA foreign_key = ON")
        con.execute("DELETE FROM {tn} WHERE {idf}={my_id}".format(tn=table, idf=pkcol, my_id=entryID))

        con.commit()

In [ ]:
import RESTful as r
r = reload(r)

In [ ]:
jFILE = "json6.json"

In [ ]:
r.entryintotables(mydb, jFILE)

In [ ]:
c = sqlCMDToPD("CONFERENCES", mydb)
t = sqlCMDToPD("ABSTRACTSTOTAL", mydb)
pap = sqlCMDToPD("PAPER", mydb)
pub = sqlCMDToPD("PUBLICATIONS", mydb)
k = sqlCMDToPD("KEYS", mydb)

au = sqlCMDToPD("AUTHORS", mydb)
aff = sqlCMDToPD("AFFILIATIONS", mydb)

pk = sqlCMDToPD("PAPERKEY", mydb)
ap = sqlCMDToPD("AFFILIATIONPAPER", mydb)
pa = sqlCMDToPD("PAPERAUTHOR", mydb)
pap.tail(2)

In [ ]:
deleteRowPK(mydb, "PAPER", "paperID", 1103)
deleteRowPK(mydb, "PAPERKEY", "paperID", 1103)

In [ ]:
c = sqlCMDToPD("CONFERENCES", mydb)
t = sqlCMDToPD("ABSTRACTSTOTAL", mydb)
pap = sqlCMDToPD("PAPER", mydb)
pub = sqlCMDToPD("PUBLICATIONS", mydb)
k = sqlCMDToPD("KEYS", mydb)

au = sqlCMDToPD("AUTHORS", mydb)
aff = sqlCMDToPD("AFFILIATIONS", mydb)

pk = sqlCMDToPD("PAPERKEY", mydb)
ap = sqlCMDToPD("AFFILIATIONPAPER", mydb)
pa = sqlCMDToPD("PAPERAUTHOR", mydb)

In [ ]:
pap.tail()

In [ ]:
def deleteRowPK(db, tab1, pkcol, entryID, tab2):
    '''Deleting a Record by PRIMARY KEY
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param pkcol str : primary column name being used, 
    param entryID int : integer value (Primary Key Value) to delete from table
    '''
    with sql.connect(db) as con:
        con.execute("PRAGMA foreign_keys = ON")
        con.execute("DELETE FROM {tn} WHERE {idf}={my_id}".format(tn=tab1, idf=pkcol, my_id=entryID))

        con.commit()

        df1  = con.execute("select * from {b} LIMIT 5".format(b = tab2)).fetchall()
        df2 =  con.execute("select * from {tn} LIMIT 5".format(tn = tab1)).fetchall()
    
    return df1, df2

In [ ]:
s = pap.query("paperID == 4")
vals = map(list, s[['abstract', 'confName', 'pubYear']].values)[0]
q = t.query('@vals[0] in Abstract').query("@vals[1] == Conf").query("@vals[2] == year")

q

In [ ]:
str(vals[0])

In [ ]:
def selectPaperID(db, paperID):
    '''
    : param NONE
    : output : Returns a json dictionary of the table names, entry counts, and links to tables 
                of all table names in the database
    ''' 
    with sql.connect(db) as con:
        c = con.cursor()
        c.execute("SELECT abstract, confName, pubYear FROM PAPER WHERE paperID = {my_id} ".format(my_id = paperID)) 
        return c.fetchone()

def deleteRowTABLE(db, paperID):
    '''Deleting a Record by PRIMARY KEY
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param pkcol str : primary column name being used, 
    param entryID int : integer value (Primary Key Value) to delete from table
    '''
    values = selectPaperID(db, paperID)
    with sql.connect(db) as con:
       
        con.execute("DELETE FROM {tn} WHERE Abstract = '%s' AND Conf= '%s' AND year = '%s'".format(tn='ABSTRACTSTOTAL')%(values[0],
                                                                                                                        values[1],
                                                                                                                        values[2]))

        df = r.sqlCMDToPD('ABSTRACTSTOTAL', db)
    
    return df

def deletebyPaper(db, myID):
    
    #delete the paper from paperTable
    r.deleteRowPK(mydb, 'PAPER', 'paperID', myID)
    #delete from composites
    r.deleteRowPK(mydb, 'PAPERKEY', 'paperID', myID)
    r.deleteRowPK(mydb, 'PAPERAUTHOR', 'paperID', myID)
    r.deleteRowPK(mydb, 'AFFILIATIONPAPER', 'paperID', myID)
    r.deleteRowPK(mydb, 'PAPER', 'paperID', myID)
    
    #delete from TOTAL ABSTRACTS
    
    deleteRowTABLE(db, myID)
    
    return df
    
    
    

In [12]:

def selectPaperID(db, paperID):
    '''
    : param NONE
    : output : Returns a json dictionary of the table names, entry counts, and links to tables 
                of all table names in the database
    ''' 
    with sql.connect(db) as con:
        c = con.cursor()
        c.execute("SELECT abstract, confName, pubYear FROM PAPER WHERE paperID = {my_id} ".format(my_id = paperID)) 
        return c.fetchone()

def deleteRowTABLE(db, paperID):
    '''Deleting a Record by PRIMARY KEY
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param pkcol str : primary column name being used, 
    param entryID int : integer value (Primary Key Value) to delete from table
    '''
    values = selectPaperID(db, paperID)
    with sql.connect(db) as con:
       
        con.execute("DELETE FROM {tn} WHERE Abstract = '%s' AND Conf= '%s' AND year = '%s'".format(tn='ABSTRACTSTOTAL')%(values[0],
                                                                                                                        values[1],
                                                                                                                        values[2]))

        df = r.sqlCMDToPD('ABSTRACTSTOTAL', db)
    
    return df

In [ ]:
deleteRowTABLE(mydb, 10)


In [ ]:
t = sqlCMDToPD('ABSTRACTSTOTAL', mydb)

In [ ]:
sql_del = """
DELETE FROM ABSTRACTSTOTAL WHERE year = {y} && Conf = '%s' AND '%s' in Abstract
""".format(y = vals[2])%(vals[1],vals[0] )

In [ ]:
def getTablesFK(db, cmd = sql_del):
    '''
    : param NONE
    : output : Returns a json dictionary of the table names, entry counts, and links to tables 
                of all table names in the database
    ''' 
    with sql.connect(db) as con:
    
        df = pd.read_sql_query(cmd, con)
        
        return df

In [13]:
selectPaperID(mydb, 18)  

NameError: name 'mydb' is not defined

In [ ]:
f, x = selectPaperID(mydb, 8)

In [ ]:
f


In [ ]:
x[0]

In [26]:
import deletionNOCASCADE as dC
dC = reload(dC)
import RESTful as r
r = reload(r)

In [28]:
c = r.sqlCMDToPD("CONFERENCES", mydb)
t = r.sqlCMDToPD("ABSTRACTSTOTAL", mydb)
pap = r.sqlCMDToPD("PAPER", mydb)
pub = r.sqlCMDToPD("PUBLICATIONS", mydb)
k = r.sqlCMDToPD("KEYS", mydb)

au = r.sqlCMDToPD("AUTHORS", mydb)
aff = r.sqlCMDToPD("AFFILIATIONS", mydb)

pk = r.sqlCMDToPD("PAPERKEY", mydb)
ap = r.sqlCMDToPD("AFFILIATIONPAPER", mydb)
pa = r.sqlCMDToPD("PAPERAUTHOR", mydb)
pap.tail()


(3, 2)
(1140, 7)
(1102, 8)
(27, 3)
(5557, 2)
(1949, 2)
(830, 2)
(17436, 2)
(1102, 2)
(3432, 2)


,paperID,abstract,title,terms,authors,affiliation,pubYear,confName
1097,1098,With the spread of the Internet and software e...,Towards a formal model for reconfigurable soft...,"['Architecture', 'Chlorine compounds', 'Digita...","['Chang, Zhiming', '', '', 'Mao, Xinjun', '', ...","Dept of Computer Science, National University...",2008,WICSA
1098,1099,With the widespread adoption of cloud computin...,A framework for architecture driven migration ...,"['Cloud computing', 'Reengineering', 'Reusabil...","['Ahmad, Aakash', '', '', 'Babar, Muhammad Ali...","IT University of Copenhagen, Denmark 2 Unive...",2014,WICSA
1099,1100,Within the business software domain it is cruc...,The role of variability patterns in multi tena...,"['Software architecture', 'Multi-tenancy', 'On...","['Kabbedijk, J', '', '', ' Jansen, S', '', '']","Information and Computing Sciences, Utrecht Un...",2012,ECSA
1100,1101,Within the business software domain it is cruc...,The role of variability patterns in multi tena...,"['Software architecture', 'Multi-tenancy', 'On...","['Kabbedijk, J', '', '', ' Jansen, S', '', '']","Information and Computing Sciences, Utrecht Un...",2012,WICSA
1101,1102,Without rigorous software development and main...,A case study in incremental architecture based...,"['Computer programming languages', 'Computer s...","['Abi Antoun, Marwan', '', '', 'Coelho, Wesley']","Institute for Software Research Intl ISRI , C...",2005,WICSA


In [29]:
dC.deletebyPaper(1102)

1102
deleted from PK
deleted from PA
deleted from AP
(u'Without rigorous software development and maintenance, software tends to lose its original architectural structure and become more difficult to understand and modify  ArchJava, a recently proposed implementation language which embeds a component and connector architectural specification within Java implementation code, offers the promise of preventing the loss of architectural structure  We describe a case study in which we incrementally re engineer an existing implementation with an eroded architecture to obtain an ArchJava implementation that more closely matches an idealized architecture  Building on results from similar case studies, we chose an application consisting of over 16,000 source lines of Java code and 80 classes that exhibited many characteristics of real world legacy applications  We describe our process, some lessons learned, as well as some perceived limitations with the tools, techniques and languages we used   

In [30]:
c = r.sqlCMDToPD("CONFERENCES", mydb)
t = r.sqlCMDToPD("ABSTRACTSTOTAL", mydb)
pap = r.sqlCMDToPD("PAPER", mydb)
pub = r.sqlCMDToPD("PUBLICATIONS", mydb)
k = r.sqlCMDToPD("KEYS", mydb)

au = r.sqlCMDToPD("AUTHORS", mydb)
aff = r.sqlCMDToPD("AFFILIATIONS", mydb)

pk = r.sqlCMDToPD("PAPERKEY", mydb)
ap = r.sqlCMDToPD("AFFILIATIONPAPER", mydb)
pa = r.sqlCMDToPD("PAPERAUTHOR", mydb)
pap.tail()


(3, 2)
(1139, 7)
(1101, 8)
(27, 3)
(5557, 2)
(1949, 2)
(830, 2)
(17424, 2)
(1101, 2)
(3430, 2)


,paperID,abstract,title,terms,authors,affiliation,pubYear,confName
1096,1097,"With the maturation of this field, companies a...",Is architecture a product that can be packaged...,"['Application software, Software engineering, ...","['Kozaczynski, W']",Rational Software Corp,2001,WICSA
1097,1098,With the spread of the Internet and software e...,Towards a formal model for reconfigurable soft...,"['Architecture', 'Chlorine compounds', 'Digita...","['Chang, Zhiming', '', '', 'Mao, Xinjun', '', ...","Dept of Computer Science, National University...",2008,WICSA
1098,1099,With the widespread adoption of cloud computin...,A framework for architecture driven migration ...,"['Cloud computing', 'Reengineering', 'Reusabil...","['Ahmad, Aakash', '', '', 'Babar, Muhammad Ali...","IT University of Copenhagen, Denmark 2 Unive...",2014,WICSA
1099,1100,Within the business software domain it is cruc...,The role of variability patterns in multi tena...,"['Software architecture', 'Multi-tenancy', 'On...","['Kabbedijk, J', '', '', ' Jansen, S', '', '']","Information and Computing Sciences, Utrecht Un...",2012,ECSA
1100,1101,Within the business software domain it is cruc...,The role of variability patterns in multi tena...,"['Software architecture', 'Multi-tenancy', 'On...","['Kabbedijk, J', '', '', ' Jansen, S', '', '']","Information and Computing Sciences, Utrecht Un...",2012,WICSA
